In [1]:
import pandas as pd
import numpy as np
import datetime

1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
loanlen = pd.read_csv("loans_lenders.csv")
loanlen.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [3]:
#creazione di un dataframe con due colonne, la prima con i lender ottenuti splittando tramite virgola quelli nel dataset di origine, 
#la seconda con gli id che si ripetono tante volte quanti sono i lender corrispondenti e sono assegnati correttamente perché l'ordine e 
#la sequenza sono mantenuti

loanlen['lenders'] = loanlen['lenders'].str.split(',')
new_loanlen = pd.DataFrame({'lender': np.concatenate(loanlen['lenders'].values),'id': loanlen['loan_id'].repeat(loanlen['lenders'].apply(len))})
new_loanlen.head()

,lender,id
0,muc888,483693
0,sam4326,483693
0,camaran3922,483693
0,lachheb1865,483693
0,rebecca3499,483693


2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [4]:
loans = pd.read_csv("loans.csv")
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [5]:
#conversione del tipo delle colonne di interesse e creazione della colonna richiesta tramite differenza tra le due considerate

loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['duration'] = loans['disburse_time'] - loans['planned_expiration_time']
loans['duration'] = loans['duration'].astype('timedelta64[D]')

3. Find the lenders that have funded at least twice.

In [6]:
#raggruppamento ed estrazione dei lender richiesti ponendo una condizione sul conteggio degli id per lender

count_lender = new_loanlen.groupby('lender').id.count().reset_index()
multiple = count_lender[count_lender['id'] > 1].dropna()
multiple.head()

,lender,id
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4


4. For each country, compute how many loans have involved that country as borrowers.

In [7]:
loans.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'activity_name', 'sector_name', 'loan_use', 'country_code',
       'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders',
       'borrower_pictured', 'repayment_interval', 'distribution_model',
       'duration'],
      dtype='object')

In [8]:
#raggruppamento e computazione di dati aggregati ai fini del calcolo richiesto

country_borrowers = loans.groupby('country_name').loan_id.count().reset_index() 
country_borrowers.head()

,country_name,loan_id
0,Afghanistan,2337
1,Albania,3075
2,Armenia,13952
3,Azerbaijan,10172
4,Belize,218


5. For each country, compute the overall amount of money borrowed.

In [9]:
#raggruppamento e computazione di dati aggregati ai fini del calcolo richiesto

money_borrowed = loans.groupby('country_name').loan_amount.sum().reset_index() 
money_borrowed.head()

,country_name,loan_amount
0,Afghanistan,1967950.0
1,Albania,4307350.0
2,Armenia,22950475.0
3,Azerbaijan,14784625.0
4,Belize,150175.0


6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [10]:
#borrow -> prendere in prestito
#lend -> dare in prestito
#loan -> denaro ricevuto in prestito
#fund -> denaro dato in prestito

#calcolo della colonna richiesta

money_borrowed['percentage_lent'] = money_borrowed['loan_amount'] / sum(money_borrowed['loan_amount']) * 100 
money_borrowed.head()

,country_name,loan_amount,percentage_lent
0,Afghanistan,1967950.0,0.166573
1,Albania,4307350.0,0.364586
2,Armenia,22950475.0,1.942589
3,Azerbaijan,14784625.0,1.251410
4,Belize,150175.0,0.012711


7. Like the three previous points, but split for each year (with respect to disburse time).

In [11]:
#estrazione dell'anno

loans['disburse_time'] = pd.to_datetime(loans['disburse_time'])
loans['year'] = pd.DatetimeIndex(loans['disburse_time']).year 

In [12]:
#creazione del dataframe richiesto tramite raggruppamento e aggiunta delle colonne per anno e Paese

new_money_borrowed = loans.groupby(['country_name','year']).agg({'loan_id': 'count', 'loan_amount': 'sum'}).reset_index()
new_money_borrowed['percentage_lent'] = new_money_borrowed['loan_amount'] / sum(new_money_borrowed['loan_amount']) * 100
new_money_borrowed.head()

,country_name,year,loan_id,loan_amount,percentage_lent
0,Afghanistan,2007.0,408,194975.0,0.016657
1,Afghanistan,2008.0,370,365375.0,0.031215
2,Afghanistan,2009.0,678,585125.0,0.049989
3,Afghanistan,2010.0,632,563350.0,0.048129
4,Afghanistan,2011.0,247,245125.0,0.020942


8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [13]:
#mantenimento delle colonne di interesse

new_loans = loans[['loan_id','loan_amount']]
new_loans.head()

,loan_id,loan_amount
0,657307,125.0
1,657259,400.0
2,658010,400.0
3,659347,625.0
4,656933,425.0


In [14]:
#unione con il dataset ricavato al punto 1 per ottenere le informazioni mancanti

join_loanlen = new_loans.join(new_loanlen.set_index('id'), on = 'loan_id')
join_loanlen.head()

,loan_id,loan_amount,lender
0,657307,125.0,spencer5657
0,657307,125.0,matthew8640
0,657307,125.0,larry71496105
1,657259,400.0,ltr
1,657259,400.0,andrew5306


In [15]:
#conteggio dei lender per ogni loan id

lender_per_loan = join_loanlen.groupby('loan_id').lender.count().reset_index()
lender_per_loan = lender_per_loan.rename(columns = {"lender": "count_lender"})
lender_per_loan.head()

,loan_id,count_lender
0,84,3
1,85,2
2,86,3
3,88,3
4,89,4


In [16]:
#integrazione del dataframe precedente con le nuove informazioni ottenute

new_join_loanlen = join_loanlen.join(lender_per_loan.set_index('loan_id'), on = 'loan_id')
new_join_loanlen.head()

,loan_id,loan_amount,lender,count_lender
0,657307,125.0,spencer5657,3
0,657307,125.0,matthew8640,3
0,657307,125.0,larry71496105,3
1,657259,400.0,ltr,7
1,657259,400.0,andrew5306,7


In [17]:
#calcolo del contributo di ciascun lender a ciascun loan id

new_join_loanlen['amount_per_lender'] = new_join_loanlen['loan_amount'] / new_join_loanlen['count_lender']
new_join_loanlen.head()

,loan_id,loan_amount,lender,count_lender,amount_per_lender
0,657307,125.0,spencer5657,3,41.666667
0,657307,125.0,matthew8640,3,41.666667
0,657307,125.0,larry71496105,3,41.666667
1,657259,400.0,ltr,7,57.142857
1,657259,400.0,andrew5306,7,57.142857


In [18]:
#calcolo dell'ammontare totale di denaro dato in prestito da ciascun lender

amount_lender = new_join_loanlen.groupby('lender').amount_per_lender.sum().reset_index()
amount_lender.head()

,lender,amount_per_lender
0,000,1672.618411
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [19]:
#raggruppamento e computazione di dati aggregati ai fini del calcolo richiesto

diff_lent_loan = loans.groupby('country_name').agg({'loan_id': 'count', 'loan_amount': 'sum', 'funded_amount': 'sum'}).reset_index()

In [20]:
#calcolo della differenza

diff_lent_loan['difference'] = diff_lent_loan['loan_amount'] - diff_lent_loan['funded_amount']
diff_lent_loan.head()

,country_name,loan_id,loan_amount,funded_amount,difference
0,Afghanistan,2337,1967950.0,1964650.0,3300.0
1,Albania,3075,4307350.0,3992475.0,314875.0
2,Armenia,13952,22950475.0,20284675.0,2665800.0
3,Azerbaijan,10172,14784625.0,14191350.0,593275.0
4,Belize,218,150175.0,150175.0,0.0


10. Which country has the highest ratio between the difference computed at the previous point and the population?


In [21]:
countries = pd.read_csv('country_stats.csv')
countries.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh


In [22]:
#integrazione delle informazioni per ottenere lo Stato di ciascun lender e calcolo del rapporto richiesto

country_loans = diff_lent_loan.join(countries.set_index('country_name'), on = 'country_name')
country_loans['diff_pop_ratio'] = country_loans['difference'] / country_loans['population']
country_loans.head()

,country_name,loan_id,loan_amount,funded_amount,difference,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,diff_pop_ratio
0,Afghanistan,2337,1967950.0,1964650.0,3300.0,AF,AFG,Asia,Southern Asia,35530081.0,35.8,0.479375,60.704,10.053750,3.55000,1870.827131,Afghanistan,0.000093
1,Albania,3075,4307350.0,3992475.0,314875.0,AL,ALB,Europe,Southern Europe,2930187.0,14.3,0.764174,77.968,14.179680,9.64681,10252.469830,Albania,0.107459
2,Armenia,13952,22950475.0,20284675.0,2665800.0,AM,ARM,Asia,Western Asia,2930450.0,32.0,0.742850,74.886,12.713218,11.29105,8189.115193,Armenia,0.909690
3,Azerbaijan,10172,14784625.0,14191350.0,593275.0,AZ,AZE,Asia,Western Asia,9827589.0,4.9,0.758528,70.896,12.650430,11.16000,16413.260330,Azerbaijan,0.060368
4,Belize,218,150175.0,150175.0,0.0,BZ,BLZ,Americas,Central America,374681.0,41.0,0.706195,70.076,12.769070,10.47000,7374.814544,Belize,0.000000


In [23]:
#estrazione dello Stato con il rapporto maggiore

country_loans[country_loans['diff_pop_ratio'] == country_loans['diff_pop_ratio'].max()]

,country_name,loan_id,loan_amount,funded_amount,difference,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,diff_pop_ratio
70,Samoa,16435,12212175.0,11508250.0,703925.0,WS,WSM,Oceania,Polynesia,196440.0,NaN,0.702,NaN,NaN,NaN,NaN,Samoa,3.58341


11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [24]:
#ricavo l'informazione degli stati non al di sotto della linea di povertà e computo quello con il rapporto (calcolato al punto precedente) 
#maggiore tra questi

country_loans['not_poor_pop'] = country_loans['population'] - country_loans['population_below_poverty_line'] * country_loans['population'] / 100
country_loans['new_diff_pop_ratio'] = country_loans['difference'] / country_loans['not_poor_pop']
country_loans[country_loans['new_diff_pop_ratio'] == country_loans['new_diff_pop_ratio'].max()]

,country_name,loan_id,loan_amount,funded_amount,difference,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,diff_pop_ratio,not_poor_pop,new_diff_pop_ratio
2,Armenia,13952,22950475.0,20284675.0,2665800.0,AM,ARM,Asia,Western Asia,2930450.0,32.0,0.74285,74.886,12.713218,11.29105,8189.115193,Armenia,0.90969,1992706.0,1.337779


12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [25]:
#conversione del tipo di 'planned_expiration_time', estrazione dell'anno, trasformazione dei dati mancanti in zero

loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'])
loans['planned_year'] = pd.DatetimeIndex(loans['planned_expiration_time']).year

loans['year'] = loans['year'].fillna(0)
loans['planned_year'] = loans['planned_year'].fillna(0)
loans['duration'] = loans['duration'].fillna(0) 

In [26]:
#creazione di un dizionario tramite doppio ciclo che verifica per ogni riga se 'planned_year' e 'year' coincidono e aggiunge come 
#chiave del dizionario gli anni che di volta in volta trova nuovi, come valore la somma della quantità di loan che va ad aggingersi a quella 
#già esistente nel caso in cui l'anno sia già nel dizionario. Se le variabili non coincidono, la quantità viene distribuita e assegnata 
#all'anno corrispondente.

years = {}
for i in range(len(loans)):
  if loans['duration'][i] <= 0 :
    if loans['planned_year'][i] == 0:
        year = loans['year'][i]
        if year not in years:
          years[year] = loans['loan_amount'][i]
        else:
          years[year] += loans['loan_amount'][i]    
    else:
        year = loans['planned_year'][i]
        if year not in years:
          years[year] = loans['loan_amount'][i]
        else:
          years[year] += loans['loan_amount'][i]
  else:
    for j in range(int(loans['duration'][i]) + 1):
      daytime = loans['disburse_time'][i] + datetime.timedelta(days = j)
      daily_amount = loans['loan_amount'][i] / loans['duration'][i]
      if daytime.year not in years: 
        years[daytime.year] = daily_amount
      else:
        years[daytime.year] += daily_amount 
years

{2014.0: 156657571.93059573,
 2015.0: 168038091.2899748,
 2012.0: 107861836.59973317,
 2016.0: 163046000.9552672,
 2010.0: 72609150.0,
 2017.0: 179146369.95263642,
 2013.0: 114632660.10500377,
 2018.0: 15809270.057464143,
 2009.0: 59689475.0,
 2011.0: 91032300.0,
 2007.0: 15446525.0,
 2006.0: 1376575.0,
 2008.0: 39423050.0,
 2005.0: 102850.0,
 0.0: 1525.0}

In [27]:
loans_amount_per_year = pd.DataFrame(years.items(), columns = ['Years', 'Loan_Amount'])
loans_amount_per_year.sort_values(by = 'Years')

,Years,Loan_Amount
14,0.0,1.525000e+03
13,2005.0,1.028500e+05
11,2006.0,1.376575e+06
10,2007.0,1.544652e+07
12,2008.0,3.942305e+07
8,2009.0,5.968948e+07
4,2010.0,7.260915e+07
9,2011.0,9.103230e+07
2,2012.0,1.078618e+08
6,2013.0,1.146327e+08
